In [1]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("/Users/adriancavallaris/Documents/train.csv")

I feel that pclass, sibsp, and parch will be significant, since pclass could determine whether wealthy individuals lived, and whether spouses, siblings, parents, or children had an effect on survived as well. I don't feel as if the other variables would have much impact on the model

In [3]:
X = pd.DataFrame()
X['sex'] = df['Sex']
X['age'] = df['Age']
X['pclass'] = df['Pclass']
X['sibsp'] = df['SibSp']
X['parch'] = df['Parch']
X['survived'] = df['Survived']

I believe that age is the most important variable, so I'll compute the average age, and I'll fill in the missing values with the average. It's not the most sophisticated model, but it's better than just removing the values.

In [4]:
averageAge = X['age'].mean()
X['age'] = X['age'].fillna(averageAge)

One hot encode Sex

In [5]:
X['sex'] = pd.get_dummies(X.sex)['female']

Assign survived as the dependent variable, and remove it from X

In [6]:
y = X['survived']
X = X.drop(['survived'], axis=1)

In [7]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X= scaler.fit_transform(X)

Hold out 20% of the data for testing

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l1', C=10)

In [10]:
model.fit(X_train, y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [12]:
print "\n\n---Logistic Model---"
#logistic AUC
logit_roc_auc = roc_auc_score(y_test, model.predict(X_test))
print "Logistic AUC = %2.2f" % logit_roc_auc
print classification_report(y_test, model.predict(X_test) )



---Logistic Model---
Logistic AUC = 0.81
             precision    recall  f1-score   support

          0       0.83      0.88      0.85       105
          1       0.81      0.74      0.77        74

avg / total       0.82      0.82      0.82       179



All in all, not bad. This model is correct over 80 percent of the time, which is much better than a random guess. I ran it without filling in the average ages, and it only managed a .75, so dealing with the missing age values resulted in a greater degree of accuracy. 